# Process a language tab to yaml v. 2.0

This notebook collects language tabs from the SAPhon [Tupian Nasal Typology Input](https://docs.google.com/spreadsheets/d/1dvXFvLIV4y84CglgjAl-ZVb09IuGazs1SzFO_UJpmnI/edit#gid=1164878023) spreadsheet and creates version 2.0 yaml output.

In [1]:
import spreadsheet
import os, re, sys
import unicodedata
import requests
from pathlib import Path
import pandas as pd
import yaml
import json
from jsonschema import validate, Draft202012Validator
from jsonschema.exceptions import ValidationError, ErrorTree
import math

# From Michael Pollack
import html_generator

downloads = Path.home() / 'Downloads'
langdir = Path('./newlangs/')
(langdir / 'json').mkdir(parents=True, exist_ok=True)
yamldir = Path('../langs')

html_json_input_folder = langdir / 'json'
synth_output_folder = Path('./en/synth_inv')
ref_output_folder = Path('./en/ref_inv')

## Get Tupian input spreadsheet lang tabs

Collect the language tabs from the input spreadsheet into a dataframe, one row per lang tab.

In [2]:
ssdf = pd.DataFrame.from_records(list(spreadsheet.langsheets.values()))
ssdf['tabname'] = list(spreadsheet.langsheets.keys())
ssdf['yaml'] = ssdf['short'] + '.yaml'
ssdf = ssdf[ssdf['include']].reset_index(drop=True).drop('include', axis='columns')
assert(~ssdf['gid'].duplicated().any())
assert(~ssdf['tabname'].duplicated().any())
ssdf

,gid,short,tabname,yaml
0,896154831,Ache,Achê,Ache.yaml
1,1490621297,Akuntsu,Akuntsú,Akuntsu.yaml
2,1053005052,AnambeC,Anambé do Cairari,AnambeC.yaml
3,566996941,Apiaka,Apiaká,Apiaka.yaml
4,1610450172,Arawete,Araweté,Arawete.yaml
...,...,...,...,...
63,1787095325,Wayoro,Wayoró,Wayoro.yaml
64,146427404,Xeta,Xetá,Xeta.yaml
65,887152938,Xipaya,Xipaya,Xipaya.yaml
66,143456252,Yuqui,Yuqui,Yuqui.yaml


## Download `.tsv` files (optional)

The next cell is optional to download all lang tabs from the spreadsheet. Set `do_download` to `True` and execute the cell to do this task. For active work on a lang tab this step is not necessary and time-consuming.

In [3]:
do_download = False
if do_download:
    for row in ssdf[ssdf['include']].itertuples():
        r = requests.get(f'{spreadsheet.puburl}/pub?gid={row.gid}&single=true&output=tsv')
        r.encoding = 'utf-8'
        with open(langdir / f'{row.short}.tsv', 'w', encoding='utf-8') as out:
            out.write(r.text)

## Function definitions

Functions used to create version 2.0 yaml, work in progress.

In [4]:
#langre = re.compile(
#    r'''
#    (?P<name>[^\[]+)
#    (?P<iso>\[[^\]]+\])?
#    ''',
#    re.VERBOSE
#)

def _clean(s):
    '''
    Clean string of extraneous markup.
    '''
    if s != None:
        s = s.strip().strip('{').strip('}').strip('[').strip(']').strip()
    return s

def split_morph_ids(mids):
    morphs = []
    if mids is None or mids == '' or mids.lower().strip() == 'none':
        return []
    for mid in spreadsheet.parse_with_delims(mids):
        try:
            d = {
                'morpheme_id': mid[0],
                'morpheme_type': mid[1],
                'underlying_form': mid[2],
#                'surface_forms': spreadsheet.parse_with_delims(mid[3].strip('{').strip('}')),
                'surface_forms': [
                    unicodedata.normalize('NFD', c.strip()) \
                        for c in mid[3].strip('{').strip('}').split(',')
                ],
                'gloss': mid[4]
            }
        except Exception as e:
            msg = f'Could not parse morpheme ids: {mid}: {e}'
            raise RuntimeError(msg)
        morphs.append(d)
    return morphs

def alloprocs(allos, procs, phoneme):
    '''
    Return zipped allophones and processes extracted from allophone list.
    '''
    allolist = [_clean(a) for a in allos.split(',')]
    proclist = [_clean(p) for p in procs.split(',')]
    mapping = {a: [] for a in allolist}

    try:
        assert(len(proclist) > 0)
    except AssertionError:
        sys.stderr.write(f'Invalid entry for phoneme {phoneme}, must have at least one allophone and process\n') 
    if (len(allolist) == 1):
        mapping[allolist[0]] = proclist
    else:
        for proc in proclist:
            m = spreadsheet.procre.match(proc)
            pgd = m.groupdict()
            try:
                phone = pgd['phone'].replace('-', '')
            except AttributeError:
                sys.stderr.write(f'Process {proc} must have an allophone prefix\n')
                continue
            procname = pgd['procsubtype'] if pgd['tag'] is None else pgd['tag'] + pgd['procsubtype']
            try:
                assert(phone in mapping.keys())
                mapping[phone].append(procname)
            except AssertionError:
                sys.stderr.write(f'{proc} does not map to an allophone\n')
    for a, proclist in mapping.items():
        if a == phoneme:
            try:
                assert(proclist == [])
            except AssertionError:
                if proclist == ['preservation']:
                    pass
                else:
                    sys.stderr.write(f'Identity allophone {a} should not have a process\n')
        else:
            try:
                assert(len(proclist) > 0)
            except AssertionError:
                sys.stderr.write(f'Non-identity allophone {a} must name at least one process\n')
    return [{'allophone': k, 'processnames': v} for k, v in mapping.items()]
 

def tsv2newyaml(tsvfile, v1):
    '''
    Make a new YAML dict from a Tupian input spreadsheet tab.
    '''
    tsvlang = spreadsheet.read_lang(tsvfile, strict=False)
    natclasses, flatnatclasses, catsymb = spreadsheet.check_natclasses(tsvlang)
    allophones, alloprocs = spreadsheet.check_allophones(tsvlang, flatnatclasses)
    morph_id_map = spreadsheet.check_morpheme_ids(tsvlang)
    spreadsheet.check_procs(tsvlang, flatnatclasses, morph_id_map, catsymb, alloprocs)
    # TODO: remainder should be per-doc (synthesis, ref)
    langdoc = {'info': {}, 'synthesis': {}, 'sources': []}
    for doc in [tsvlang['synthesis']] + tsvlang['ref']:
        is_synthesis = 'synthesis' in doc.keys()
#        doc = tsvlang['synthesis']
#        if 'lang' not in doc.keys():
#            return (None, None, None)
        try:
            notes = doc['notes']
        except KeyError:
            notes = 'None'
        if is_synthesis:
            name = _clean(doc['lang'])
            if '[' in name:
                sys.stderr.write(f'Language name for {name} should be cleaned up.\n')
            try:
                glottocode = _clean(doc['glottocode'])
            except KeyError:
                sys.stderr.write(f'Lang {name} missing glottocode field.\n')
                glottocode = TODO
            try:
                altnames = [_clean(c) for c in doc['altnames'].split(',')]
            except KeyError:
                sys.stderr.write(f'Lang {name} missing alternate_names field.\n')
                altnames = [TODO]
            try:
                iso_codes = [_clean(c) for c in doc['iso_codes'].split(',')]
            except KeyError:
                sys.stderr.write(f'Lang {name} missing iso_codes field.\n')
                iso_codes = [TODO]
            if v1 == {}:
                v1 = {
                    'info': {
                        'short_name': 'TODO',
                        'alternate_names': 'TODO',
                        'family': 'TODO',
                        'countries': ['TODO'],
                        'coordinates': [],
                    },
                    'synthesis': {
                        'tone': False,
                        'laryngeal_harmony': False
                    }
                }
            try:
                langdoc['info'] = {
    #                'doctype': 'synthesis',
                      'name': name,
                      'short_name': v1['info']['short_name'],
                      'alternate_names': altnames,
                      'glottolog_code': glottocode,
                      'iso_codes': iso_codes,
                      'family': v1['info']['family'],
                      'countries': v1['info']['countries'],
                      'coordinates': v1['info']['coordinates'],
                      'notes': 'None'
                }
                langdoc['synthesis'] = {
                      'summary': doc['synthesis'],
                      'notes': notes,
                      'natural_classes': [{'symbol': nc[0], 'members': nc[1:]} for nc in natclasses['synthesis']],
                      'phonemes': phonlist(allophones['synthesis']),
                      'morphemes': split_morph_ids(doc['morph_ids']),
                      'processdetails': doc['processes'],
                 # TODO: following from v1 and not mentioned in new YAML draft
    #!            'allophones': v1['allophones'],
    #!            'nasal_harmony': v1['nasal_harmony'],
                      'tone': v1['synthesis']['tone'],
                      'laryngeal_harmony': v1['synthesis']['laryngeal_harmony'],
                }
            except Exception as e:
                sys.stderr.write(f'Problem with doc {doc}.\n')
                raise e
        else:
            langdoc['sources'].append(
                {
                      'summary': doc['summary'],
                      'notes': notes,
                      'citation': [doc['source']],
                      'natural_classes': [{'symbol': nc[0], 'members': nc[1:]} for nc in natclasses[doc['source']]],
                      'phonemes': phonlist(allophones[doc['source']]),
                      'morphemes': split_morph_ids(doc['morph_ids']),
                      'processdetails': doc['processes'],
                 # TODO: following from v1 and not mentioned in new YAML draft
    #!            'allophones': v1['allophones'],
    #!            'nasal_harmony': v1['nasal_harmony'],
                }
            )
# Filter None values out of list values.
#!    listflds = (
#!        'alternate_names', 'iso_codes',
#!        'countries', 'coordinates', 'natural_classes',
#!        'morphemes', 'phonemes', 'processes',
#!        'triggers', 'notes'
#!    )
#!    for fld in listflds:
#!        sdoc[fld] = [v for v in sdoc[fld] if v is not None]
    return (langdoc, tsvlang, allophones)

def proclist_old(processes):
    '''
    Return a `processdetails` list from spreadsheet `processes` section.
    '''
    deets = []
    for proc in processes:
        procname = proc['proc_name'] if not '-' in proc['proc_name'] else proc['proc_name'][proc['proc_name'].index('-')+1:]
        deet = {
            'processname': procname,
            'processtype': proc['proc_type'],
            'description': proc['description'],
            'optionality': proc['optionality'],
            'directionality': proc['directionality'],
            'alternation_type': proc['alternation_type']
        }
        for fld in 'undergoers', 'triggers':
            if proc[fld] == 'NA':
                deet[fld] = [['NA'], {'type': 'TODO', 'positional_restrictions': 'TODO'}]
            elif isinstance(proc[fld], dict):
                data = proc[fld][fld]
            else:
                print(f'PROC: {proc}')
                data = proc[fld][0][fld]
            try:
                deet[fld] = [
                    [_clean(f) for f in data.split(',')],
                    {
                        'type': 'TODO',
                        'positional_restrictions': 'TODO'
                    }
                ]
            except:
                print(f'failed for {fld}: "{proc[fld]}"')
        for new, old in ('transparent', 'transparencies'), ('opaque', 'opacities'):
            deet[new] = proc[old][old]
        deets.append(deet)
    return deets

def _clean_procname(p):
    if p is None:
        return 'TODO: got None'
    else:
        return _clean(p if '-' not in p else p[p.index('-')+1:])

def phonlist(allophones):
    '''
    Return a `phonemes` list from an `allophone` set.
    '''
    phonemes = {}
    for pset in allophones:
        d = {}
        if len(pset) == 5:
            seg = pset[0]
            envs = {'phoneme': f'STRING MAPPING: "{pset}"'}
            print(f'STRING MAPPING: "{pset}"')
            continue
        elif len(pset) == 4:
            phoneme, allos, _psetenv, proc = pset
            phoneme = unicodedata.normalize('NFD', phoneme)
            allophones = alloprocs(allos, proc, phoneme)
            envs = []
            for env in spreadsheet.parse_env(pset[2]):
                try:
                    env['allophones'] = allophones
                except:
                    print(env)
                envs.append(env)
        elif len(pset) == 2:
            phoneme, allos = pset
            phoneme = unicodedata.normalize('NFD', phoneme)
            envs = [{
                'preceding': '', # TODO: NA or other empty value here?
                'following': '', # TODO: NA or other empty value here?
                'allophones': [
                    {
                        'processnames': [],
                        'allophone': unicodedata.normalize('NFD', _clean(a))
                    } for a in allos.split(',')
                ]
            }]
        else:
            sys.stderr.write(f'Unexpected phoneme set length for "{pset}".')
            continue
        try:
            phonemes[phoneme]['environments'] += envs
        except KeyError:
            phonemes[phoneme] = {'phoneme': phoneme, 'environments': envs}
    # TODO: sort phonemes
    return list(phonemes.values())

def yaml2newyaml(v1):
    '''
    Copy a version 1 YAML dict into version 2.
    '''
    sdoc = {
#        'doctype': 'synthesis',
        'info': {
          'name': v1['name'],
          'short_name': v1['short_name'],
          'alternate_names': v1['alternate_names'],
#          'glottolog_code': v1['name'], # TODO: new, check by hand
          'iso_codes': v1['iso_codes'],
#          'glottolog_codes': [], # TODO: new, need to be added by hand
          'family': v1['family'],
          'countries': v1['countries'],
          'coordinates': v1['coordinates'],
          'notes': v1['notes'], # TODO: not mentioned in new YAML draft
        },
        'synthesis': {
          'natural_classes': [], # TODO: new
          'morphemes': [], # TODO: new?
          'phonemes': [{'phoneme': p} for p in v1['phonemes']],
          'processdetails': [], # TODO: new?
          'triggers': [], # TODO: new?
          'transparent': [], # TODO: new?, include?
          'opaque': [], # TODO: new?, include?
         # TODO: following from v1 and not mentioned in new YAML draft
          'allophones': v1['allophones'],
          'nasal_harmony': v1['nasal_harmony'],
          'tone': v1['tone'],
          'laryngeal_harmony': v1['laryngeal_harmony'],
        }
    }
    # Filter None values out of list values.
    infolistflds = (
        'alternate_names', 'iso_codes', 
        'countries', 'coordinates', 'notes'
    )
    for fld in infolistflds:
        sdoc['info'][fld] = [v for v in sdoc['info'][fld] if v is not None]
    synthlistflds = ('natural_classes',
        'morphemes', 'phonemes', 'processdetails',
        'triggers'
    )
    for fld in synthlistflds:
        sdoc['synthesis'][fld] = [v for v in sdoc['synthesis'][fld] if v is not None]
    return sdoc

TODO = 'TODO'

def ss2refdoc(lang):
    pass

def ss2synthdoc(lang):
    '''
    Produce a synthesis yaml doc from a lang from the input spreadsheet.
    '''
    synth = lang['synthesis']
    langm = re.match(langre, synth['lang'])
    yd = {
        'doctype': 'synthesis',
        'name': langm['name'].strip(),
        'short_name': TODO,
        'alternate_names': TODO,
        'iso_codes': langm['iso']
    }
    return yd

In [5]:
# This is just a debugging cell

sample = '''{i.3p, prefix, i, {j, ɲ}, 3rd person}, {pá, stem, pá, má, US}, {potá, stem, potá, motá, US}, {ɣʷá, postposition, ɣʷá, ŋʷá, US}, {ɣʷaré, postposition, ɣʷaré, ŋʷaré, US}, {tɨ́, suffix, tɨ́, nɨ́, US}, {pɨ́, suffix, pɨ́, mɨ́, US}, {kʷéra, stem, kʷéra, ŋʷéra, US}, {ɣʷivé, stem, ɣʷivé, ŋʷivé, US}, {pukú, stem, pukú, mukú, US}, {ɣʷasú, stem, ɣʷasú, ŋʷasú, US}, {kʷé, der.affix, kʷé, ŋʷé, US}, {pe, postposition, pe, {mẽ, ʋe, ʋ̃ẽ}, US}, {ʔó, der.affix, ʔó, ʔṍ, US}, {sé, der.affix, sé, sẽ́, US}, {ɣʷi, postposition, ɣʷi, ɣ̃ʷ̃ĩ, conjunction}, {ke, stem, ke, kẽ, modifier}, {ko, stem, ko, kõ, pronoun}, {ku, stem, ku, kũ, article}, {la, stem, la, l̃ã, article}, {na...i, stem, na...i, nã...ĩ, modifier}, {pa, stem, pa, pã, clause marker}, {ʃa, postposition, ʃa, ʃã, NA}, {ta, stem, ta, tã, modifier}, {tei, stem, tei, tẽĩ, modifier}, {ʋa, stem, ʋa, ʋ̃ã, nominalizer}, {ʋo, stem, ʋo, ʋ̃õ, conjunction}, {xa, stem, xa, xã, coordinator}, {pikó, stem, pikó, {pi, pĩ}, clause marker}, {rexé, postposition, rexé, {re, r̃ẽ}, NA}, {riré, postposition, riré, {ri, r̃ĩ}, conjunction} '''
#split_morph_ids(tsvlang['synthesis']['morph_ids'])
split_morph_ids(sample)
#print([f'{t["surface_forms"]}' for t in split_morph_ids(sample)])
#spreadsheet.parse_with_delims('{j, ɲ}')

[{'morpheme_id': 'i.3p',
  'morpheme_type': 'prefix',
  'underlying_form': 'i',
  'surface_forms': ['j', 'ɲ'],
  'gloss': '3rd person'},
 {'morpheme_id': 'pá',
  'morpheme_type': 'stem',
  'underlying_form': 'pá',
  'surface_forms': ['má'],
  'gloss': 'US'},
 {'morpheme_id': 'potá',
  'morpheme_type': 'stem',
  'underlying_form': 'potá',
  'surface_forms': ['motá'],
  'gloss': 'US'},
 {'morpheme_id': 'ɣʷá',
  'morpheme_type': 'postposition',
  'underlying_form': 'ɣʷá',
  'surface_forms': ['ŋʷá'],
  'gloss': 'US'},
 {'morpheme_id': 'ɣʷaré',
  'morpheme_type': 'postposition',
  'underlying_form': 'ɣʷaré',
  'surface_forms': ['ŋʷaré'],
  'gloss': 'US'},
 {'morpheme_id': 'tɨ́',
  'morpheme_type': 'suffix',
  'underlying_form': 'tɨ́',
  'surface_forms': ['nɨ́'],
  'gloss': 'US'},
 {'morpheme_id': 'pɨ́',
  'morpheme_type': 'suffix',
  'underlying_form': 'pɨ́',
  'surface_forms': ['mɨ́'],
  'gloss': 'US'},
 {'morpheme_id': 'kʷéra',
  'morpheme_type': 'stem',
  'underlying_form': 

In [6]:
# This is just a debugging cell

samplestrs = [
#    'p, p',
#    'e, {e, ɛ}, @, ɛ-lowering',
#    's, {s, tʃ, ʃ}, {_i, i_}, {tʃ-fortition, tʃ-palatalization, ʃ-palatalization}',
#    't, n, _+Ṽ, XMP=LN:t',
#    "e, {eː, ɛː}, $'_, {eː-lengthening, ɛː-lowering, ɛː-lengthening}",
#    'o, {o,ŏ}, _{h, ɾ}o, ŏ-vowel shortening',
    "o, ɑ, {$_$'{ʔɔ, hɔ},w_}, lowering",
    'a, {ə̃, ã}, {_{N, ND}, N_}, {ə̃-LN:V, ã-LN:V}'
]
for s in samplestrs:
    pset = spreadsheet.split_outside_delims(s)
    if len(pset) == 4:
        print(pset)
        phoneme, allos, env, proc = pset
        mapping = alloprocs(allos, proc, phoneme)
        print(mapping)

['o', 'ɑ', "{$_$'{ʔɔ, hɔ},w_}", 'lowering']
[{'allophone': 'ɑ', 'processnames': ['lowering']}]
['a', '{ə̃, ã}', '{_{N, ND}, N_}', '{ə̃-LN:V, ã-LN:V}']
[{'allophone': 'ə̃', 'processnames': ['LN:V']}, {'allophone': 'ã', 'processnames': ['LN:V']}]


## Read `.tsv` and v1 `.yaml` files

Download, read, and process lang tabs (and existing v. 1 yaml files, if they exist). Set one or more tab indexes in `rng` to be checked for errors. Set `use_cached` to `True` if you want to use a previously-downloaded `.tsv` file instead of downloading from the input spreadsheet.

In [7]:
nosynthesis = [
    12,
    21,
    25,
    32,
    35,
    37,
    43,
    44,
    47,
    48,
    50, # Siriono
    51,
    54, # Tapirapé
    57,
    63,
    65,
    67
]

In [8]:
with open('../saphonlang.schema.json', 'r', encoding='utf8') as sh:
    schema = json.load(sh)
validator = Draft202012Validator(schema)

use_cached = True
langs = {}
rng = [n for n in range(0, 67) if n not in nosynthesis]
print(f'Working on {len(rng)} lang tabs')
myerror = None
for row in ssdf.iloc[rng].itertuples():
    if (yamldir / row.yaml).exists():
        with open(yamldir / row.yaml, 'r', encoding='utf-8') as fh:
            v1docs = list(yaml.safe_load_all(fh))
            if math.isnan(v1docs[0]['coordinates'][0]['elevation_meters']):
                v1docs[0]['coordinates'][0]['elevation_meters'] = 'Unspecified'
        v1synth = yaml2newyaml(v1docs[0])
    else:
        v1synth = {}
    
    tsvfile = langdir / f'{row.short}.tsv'
    if use_cached is not True or not tsvfile.exists():
        print(f"Requesting '{row.tabname}' lang tab from index {row.Index} and caching at {tsvfile}.")
        r = requests.get(f'{spreadsheet.puburl}/pub?gid={row.gid}&single=true&output=tsv')
        r.encoding = 'utf-8'

        with open(tsvfile, 'w', encoding='utf-8') as out:
            # Replace Windows CRLF with Unix LF
            text = r.content.replace(b'\r\n', b'\n').decode('utf8')
            out.write(text)
    else:
        print(f"Reading from cached file {tsvfile} ({row.Index}).")
    try:
        v2synth, tsvlang, allophones = tsv2newyaml(tsvfile, v1synth)
        if tsvlang == None:
            sys.stderr.write(f'Skipping .json creation for {row.tabname}.\n\n')
            continue
    except Exception as e:
        v2synth = {}
        print(f'\n\nERROR: spreadsheet tab {row.tabname} failed.\n{e}\n\n')
#        raise e
        continue
    langs[row.short] = {
        'v1synth': v1synth,
        'v2synth': v2synth,
        'tsv': tsvlang['synthesis']
    }
    try:
        langjson = langdir / 'json' / f'{row.short}.json'
        with open(langjson, 'w', encoding='utf8') as out:
            json.dump(langs[row.short]['v2synth'], out, indent=2, ensure_ascii=False)
        print(f'Dumped json {row.short}.json')
        
    except Exception as e:
        sys.stderr.write(f'Failed to dump json {row.short}.json.\n')
    try:
        with open(langjson, 'r', encoding='utf8') as jh:
            langobj = json.load(jh)
#            tree = ErrorTree(validator.iter_errors(langobj))
            validate(instance=langobj, schema=schema)
    except ValidationError as e:
        myerror = e
        print(f'In json file {langjson}\njson path {e.json_path}\ngot value "{e.instance}"\nexpected {e.validator_value}')
#        break

Working on 51 lang tabs
Requesting 'Achê' lang tab from index 0 and caching at newlangs\Ache.tsv.


Allophone processname 'maintenance' does not match available Process names 'BN, BO, GN, LDNH, LDOH, LN, LNsyll, LO, MPP, SN, SO, XMP, XWP, affricate unreleasing, affrication, backing, bilabialization, centralization, consonant aspiration, consonant epenthesis, debuccalization, degemination, degliding, delabialization, deletion, dentalization, depalatalization, develarization, devoicing, fortition, fronting, gemination, gliding, glottal constriction, glottalic initiation, hnasalization, labialization, labiodentalization, lateralization, lengthening, lenition, lowering, metathesis, nasal coda neutralization, nasal consonant postglottalization, nasal place assimilation, nasal unreleasing, nasalization, onset fronting, oral consonant postglottalization, oralization, palatalization, PNV, preservation, raising, representation convention, restricted UR realization, retroflexing, rhotacization, rounding, semivocalization, spirantization, tapping, tone raising, total assimilation, trilling, UC,

Dumped json Ache.json
In json file newlangs\json\Ache.json
json path $.synthesis.processdetails[4].undergoers.segments.positional_restrictions
got value "{root+suffix}"
expected [{'type': 'string', 'pattern': '^\\{(root|prefix|suffix|proclitic|enclitic|clitic|word|morpheme|affix|syllable|foot|stressed\\ syllable|unstressed\\ syllable|pretonic\\ syllable|posttonic\\ syllable|utterance|VP)(,\\s*(initial|final|lmost|rmost|medial|onset|coda|nucleus))?\\}(\\s*(,|>)\\s*\\{(root|prefix|suffix|proclitic|enclitic|clitic|word|morpheme|affix|syllable|foot|stressed\\ syllable|unstressed\\ syllable|pretonic\\ syllable|posttonic\\ syllable|utterance|VP)(,\\s*(initial|final|lmost|rmost|medial|onset|coda|nucleus))?\\})*$', '$comment': 'This pattern allows for controlled vocabulary in positional restrictions. `foo` is restricted to linguistic elements, and `bar` is restricted to positional terms.'}, {'type': 'string', 'enum': ['Unspecified', 'Uncertain', 'None', 'NA'], '$comment': 'Special vocabulary t

Process backing must have an allophone prefix
Non-identity allophone ɐ̃ must name at least one process
Non-identity allophone ɔ̃ must name at least one process
Non-identity allophone ʌ̃ must name at least one process
Process glottal constriction must have an allophone prefix
Non-identity allophone ḛ must name at least one process
Non-identity allophone ɛ̰ must name at least one process
Process glottal constriction must have an allophone prefix
Non-identity allophone o̰ must name at least one process
Non-identity allophone ṵ must name at least one process
Process glottal constriction must have an allophone prefix
Non-identity allophone ḛ̃ must name at least one process
Non-identity allophone ɛ̰̃ must name at least one process
Process glottal constriction must have an allophone prefix
Non-identity allophone õ̰ must name at least one process
Non-identity allophone ṵ̃ must name at least one process


Dumped json Akuntsu.json
In json file newlangs\json\Akuntsu.json
json path $.synthesis.processdetails[4].undergoers.segments.positional_restrictions
got value "{morpheme1, final}"
expected [{'type': 'string', 'pattern': '^\\{(root|prefix|suffix|proclitic|enclitic|clitic|word|morpheme|affix|syllable|foot|stressed\\ syllable|unstressed\\ syllable|pretonic\\ syllable|posttonic\\ syllable|utterance|VP)(,\\s*(initial|final|lmost|rmost|medial|onset|coda|nucleus))?\\}(\\s*(,|>)\\s*\\{(root|prefix|suffix|proclitic|enclitic|clitic|word|morpheme|affix|syllable|foot|stressed\\ syllable|unstressed\\ syllable|pretonic\\ syllable|posttonic\\ syllable|utterance|VP)(,\\s*(initial|final|lmost|rmost|medial|onset|coda|nucleus))?\\})*$', '$comment': 'This pattern allows for controlled vocabulary in positional restrictions. `foo` is restricted to linguistic elements, and `bar` is restricted to positional terms.'}, {'type': 'string', 'enum': ['Unspecified', 'Uncertain', 'None', 'NA'], '$comment': 'Special v

...

STRING MAPPING: "['m', '{m, mb}', '/mV/', '{mṼ, mbV}', 'mb-LO:mn']"
STRING MAPPING: "['n', '{n, nd}', '/nV/', '{nṼ, ndV}', 'nd-LO:mn']"
STRING MAPPING: "['V', '{V, Ṽ}', '/mV/', '{mṼ, mbV}', 'Ṽ-LN:rightward']"
STRING MAPPING: "['V', '{V, Ṽ}', '/nV/', '{nṼ, ndV}', 'Ṽ-LN:rightward']"
Dumped json Kaapor.json
In json file newlangs\json\Kaapor.json
json path $.sources[0].processdetails[3].directionality
got value "uncertain"
expected ['leftward', 'rightward', 'bidirectional', 'circumdirectional', 'unknown', 'NA']
Requesting 'Warázu' lang tab from index 61 and caching at newlangs\Warazu.tsv.


Process name LNsyll does not match LNSyll for synthesis

Process name LNsyll does not match LNSyll for Ramirez, Henri, Valdir Vegini, & Maria Cristina Victorino de França. 2017. O warázu do Guaporé (tupi-guarani): primeira descrição linguística. LIAMES 17. 411-506.



Dumped json Warazu.json
Requesting 'Wayampi' lang tab from index 62 and caching at newlangs\Wayampi.tsv.
top: Process name:	MPP=LNsyll:pɛ
Process type:	LNsyll
Prose description:	pɛ is nasalized into mɛ̃ following a root that ends with a nasal vowel. 
Page numbers:	48, 76, 77
Optionality:	Categorical
Directionality:	Rightward
Alternation type:	Phonological

bottom: Undergoers:	p, ɛ
Type:	Morphophonological
Morpheme class:	{postposition}
Morpheme IDs:	pɛ
Positional restriction:	{postposition, initial}
Triggers:	Ṽ
Type:	Segmental
Morpheme class:	{root}
Morpheme IDs:	NA
Positional restriction:	{root, final}
Transparencies:	NA
Opacities:	NA
gd: {'fld': 'Undergoers', 'objs': 'p, ɛ', 'type': 'Morphophonological', 'mclass': '{postposition}', 'mids': 'pɛ', 'posres': '{postposition, initial}', 'tos': None}
SECTION s: Process name:	MPP=LNsyll:pɛ
Process type:	LNsyll
Prose description:	pɛ is nasalized into mɛ̃ following a root that ends with a nasal vowel. 
Page numbers:	48, 76, 77
Optionality:	C

Process LO:neutralization must have an allophone prefix
Non-identity allophone ɾ must name at least one process
Non-identity allophone d must name at least one process
Process LO:neutralization must have an allophone prefix
Non-identity allophone ʒ must name at least one process
Non-identity allophone j must name at least one process


In json file newlangs\json\Yuqui.json
json path $.synthesis.morphemes[0].surface_forms[0]
got value "NA"
expected ^(p|pː|pʲ|pʷ|ps|ʰb|b|bː|bʲ|bʷ|bz|bβ|bʰ|t̪|t̪ʙ|d̪|t|tˣ|ʰt|tː|tʲ|tʷ|t̺|ʰd|d|dː|dʲ|ʈ|ɖ|c|ʰc|ʰɟ|ɟ|k|kʲ|kˣ|ʰk|kː|ʰkʷ|kʷ|kʷː|k̪|kʷʲ|kp|tk|ɡ|ɡː|ɡʷ|q|qʷ|ɢ|ʔ|ʔʲ|ts|tsʲ|dz|nð|tʃ|tʃʲ|dʒ|dʃ|ʈʂ|ɖʐ|tɕ|cç|cçʰ|ɟʝ|ɓ̥|ɓ|ɗ̥|ɗ|ʄ|ɠ|pʰ|pʲʰ|t̪ʰ|d̪ʰ|tʰ|tʲʰ|dʰ|ʈʰ|cʰ|kʰ|kʲʰ|qʰ|tsʰ|tsʲʰ|tʃʰ|ʈʂʰ|pʼ|b̰|ʔb|t̪ʼ|tʼ|tʲʼ|t̰|t◌̰|d̰|ʔd|ʈʼ|cʼ|c̰|ɟ̰|kʼ|kʲʼ|kʷʼ|k̰|kʷ̰|kʷ◌̰|ɡ̰|qʼ|tsʼ|ʔdʒ|tʃʼ|ʈʂʼ|mp|mpʲ|mɸʲ|mb|mbʲ|nt|ntʲ|ns|nsʲ|nd|ndʲ|ŋk|ŋɡ|ŋɡʷ|ntʃ|ndʒ|ɳʈʂ|ɳɖʐ|nts|ndz|ndzʲ|ʰm|mʰ|m̥|m|mʼ|m̰|mʲ|mʷ|mː|ɱ|n̪|ʰn|nʰ|n̥|n|nʲ|nː|n̰|ɲʰ|ʰɲ|ɲ|ɲ̰|ɲː|ɲ̥|ŋ|ŋʷ|ŋ̥|ɴ|N|ɸ|ɸʲ|β|βʲ|fʷ|f|v|θ|s̪|θʲ|ð|s|sʲ|sʼ|sʰ|s̺|z|ɼ|ʂ|ʐ|ʃ|ʒ|ʒ̺|ɕ|ç|ʝ|x|xʲ|xl|xʷ|ɣ|ɣʷ|χ|χʷ|ʁ|h|h̃|hʲ|hʷ|ɦ|ʍ|β̞|ʋ|ð̞|ɹ|ɻ|ɻ̥|ʰj|j|j̃|j̥|jʼ|j̰|jː|ɰ|ʰw|wʰ|w̥|w|w̃|wʼ|w̰|wʲ|wː|ʔw|ʕ|l̪|l̥|ɬ|ɺ̥|l|lʲ|lʼ|lː|ɺ|ɮ|ɭ|ɭʲ|ʎ|kl|ʰɾ|ɾ|ɾ̃|ɾʲ|ʔɾ|ɽʰ|ɽ|ⱱ|r̥|r|ʀ|ʙ|i|ĩ|ḭ|ḭ̃|iː|ĩː|iʰ|iˀ|ĭ|ĭ̃|y|ỹ|ɨ|ɨ̃|ɨ̰|ɨː|ɨ̃ː|ɨ̘|ɨi|ɨ̆|ɨ̆̃|ʉ|ʉ̃|ʉː|ʉ̃ː|ɯ|ɯ̃|ɯ̰|ɯː|ɯ̃ː|ɯi|u|ũ|ṵ|ṵ̃|uː|ũː|uʰ|uˀ|ŭ|ŭ̃|ɪ|ɪ̃|ɪː

In [9]:
# Run the script to process all template files in the json folder and save the HTML files in the output folder
print(f'Generating html files from json files in {langdir}')
html_generator.process_templates_from_folder(
    html_json_input_folder, synth_output_folder, ref_output_folder
)
print(f'Generated synthesis html files in {synth_output_folder}')
print(f'Generated reference html files in {ref_output_folder}')

Generating html files from json files in newlangs
Working on newlangs\json\Ache.json
Generated Synthesis HTML file: en\synth_inv\Ache.html
Generated Reference HTML file: en\ref_inv\Ache.html
Working on newlangs\json\Akuntsu.json
Generated Synthesis HTML file: en\synth_inv\Akuntsu.html
Generated Reference HTML file: en\ref_inv\Akuntsu.html
Working on newlangs\json\AnambeC.json
Generated Synthesis HTML file: en\synth_inv\AnambeC.html
Generated Reference HTML file: en\ref_inv\AnambeC.html
Working on newlangs\json\Apiaka.json
Generated Synthesis HTML file: en\synth_inv\Apiaka.html
Generated Reference HTML file: en\ref_inv\Apiaka.html
Working on newlangs\json\AsuriniT.json
Generated Synthesis HTML file: en\synth_inv\AsuriniT.html
Generated Reference HTML file: en\ref_inv\AsuriniT.html
Working on newlangs\json\AsuriniX.json
Generated Synthesis HTML file: en\synth_inv\AsuriniX.html
Generated Reference HTML file: en\ref_inv\AsuriniX.html
Working on newlangs\json\AvaCanoeiroG.json
Generated Syn

### Extra stuff

For debugging, etc.

In [ ]:
langjson = langdir / 'json' / 'Ache.json'
with open(langjson, 'r') as jh:
    lang = json.load(jh)

In [ ]:
info = lang['info']
coordinates = info['coordinates']
coordinates

In [ ]:
langjson, myerror.message, myerror.json_path, myerror.absolute_path, myerror.instance

In [ ]:
v = Draft202012Validator(schema)
for error in sorted(v.iter_errors(langobj), key=str):
    print(error.message)